In [ ]:
# if you run this notebook from google colab, run this cell
!git clone https://github.com/scussatb/pyCGP.git
%cd pyCGP

In [2]:
from pycgp import CGP, CGPES, Evaluator
from pycgp.cgpfunctions import *
import pandas as pd
import sys
import time

# CGP and evolution parameters

In [10]:
col=30
row=1
nb_ind=5
mutation_rate_nodes=0.15
mutation_rate_outputs=0.3
n_it=10000
genome=None

# Function lib

In [11]:
def build_funcLib():
    return [CGP.CGPFunc(f_sum, 'sum', 2, 0),
            CGP.CGPFunc(f_aminus, 'aminus', 2, 0),
            CGP.CGPFunc(f_mult, 'mult', 2, 0),
            CGP.CGPFunc(f_exp, 'exp', 2, 0),
            CGP.CGPFunc(f_abs, 'abs', 1, 0),
            CGP.CGPFunc(f_sqrt, 'sqrt', 1, 0),
            CGP.CGPFunc(f_sqrtxy, 'sqrtxy', 2, 0),
            CGP.CGPFunc(f_squared, 'squared', 1, 0),
            CGP.CGPFunc(f_pow, 'pow', 2, 0),
            CGP.CGPFunc(f_one, 'one', 0, 0),
            CGP.CGPFunc(f_zero, 'zero', 0, 0),
#            CGP.CGPFunc(f_const, 'const', 0, 1),
            CGP.CGPFunc(f_inv, 'inv', 1, 0),
            CGP.CGPFunc(f_gt, 'gt', 2, 0),
            CGP.CGPFunc(f_asin, 'asin', 1, 0),
            CGP.CGPFunc(f_acos, 'acos', 1, 0),
            CGP.CGPFunc(f_atan, 'atan', 1, 0),
            CGP.CGPFunc(f_min, 'min', 2, 0),
            CGP.CGPFunc(f_max, 'max', 2, 0),
            CGP.CGPFunc(f_round, 'round', 1, 0),
            CGP.CGPFunc(f_floor, 'floor', 1, 0),
            CGP.CGPFunc(f_ceil, 'ceil', 1, 0)
           ]

library = build_funcLib()

# Dataset infos

In [12]:
class SinEvaluator(Evaluator):
    def __init__(self):
        super().__init__()
        self.n_inputs = 1
        self.n_outputs = 1

    def evaluate(self, cgp, it, displayTrace = False):
        fit = 0.0
        x = 0.0
        while x < 1.0:
            cgpVal =  cgp.run([x])[0]
            toFit = np.sin(x)
            fit += abs(toFit - cgpVal)
            if displayTrace:
                print(x, '\t', toFit, '\t', cgpVal)
            x += 0.01
        return -fit

    def clone(self):
        return SinEvaluator()
dirname = 'datasets/sinwave'
e = SinEvaluator()

# Starting evolution

In [14]:
if genome is None:
    cgpFather = CGP.random(num_inputs=e.n_inputs, num_outputs=e.n_outputs, 
                           num_cols=col, num_rows=row, library=library, 
                           recurrency_distance=1.0, recursive=False, 
                           const_min=-1, const_max=1.0,
                           input_shape=1, dtype='float32')
else:

    
    cgpFather = CGP.load_from_file(genome, library)

output_dirname = dirname+'/evos/run_'+str(round(time.time() * 1000000))
print("Starting evolution. Genomes will be saved in: "+ output_dirname)
es = CGPES(nb_ind, mutation_rate_nodes, mutation_rate_outputs, cgpFather, e, output_dirname)
es.run(n_it, print_every=100)

Starting evolution. Genomes will be saved in: datasets/diabetes/evos/run_1748346365841970
100 	 -0.23304641104981 	 True 	 [-0.23304641 -0.27341386 -0.23304641 -0.23304641 -0.23304641]
200 	 -0.23304641104981 	 True 	 [-0.27341386 -0.23304641 -0.23304641 -0.23304641 -0.23304641]
300 	 -0.23304641104981 	 True 	 [-0.23304641 -0.23304641 -0.27341386 -0.37880869 -0.23304641]
400 	 -0.16795172677634526 	 True 	 [-0.16795173 -0.16795173 -0.16795173 -0.27341386 -0.27341386]
500 	 -0.16795172677634526 	 True 	 [-0.16795173 -0.16795173 -0.16795173 -0.16795173 -0.16795173]
600 	 -0.16795172677634526 	 True 	 [-0.16795173 -0.16795173 -0.16795173 -0.16795173 -0.16795173]
700 	 -0.16795172677634526 	 True 	 [-0.16795173 -0.16795173 -0.16795173 -0.16795173 -0.27341386]
800 	 -0.16795172677634526 	 True 	 [-0.16795173 -0.16795173 -0.16795173 -0.16795173 -0.16795173]
900 	 -0.16795172677634526 	 True 	 [-0.27341386 -0.16795173 -0.27341386 -0.27341386 -0.27341386]
1000 	 -0.16795172677634526 	 True 	 

# Display and evaluation of the best individual

In [15]:
es.father.to_function_string([input_name for input_name in input_names], ['y_'+str(i) for i in range(e.n_outputs)])


'y_0 = sum(s5, bmi);'

In [16]:
e.evaluate(es.father, 0, True)

-0.12628842149488284

In [17]:
import os
es.father.to_dot('cgp_graph.dot', ['input_'+str(i) for i in range(e.n_inputs)], ['output_'+str(i) for i in range(e.n_outputs)])
#os.system('dot -Tpdf cgp_graph.dot -o cgp_graph.pdf')
#os.system('rm cgp_graph.dot')

render the graph with [this site](https://dreampuf.github.io/GraphvizOnline)